In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
heart = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
heart.head()

In [ ]:
heart.info()

The dataset appears to have no null values

In [ ]:
heart.describe()

In [ ]:
heart.columns

---

# Exploratory Data Analysis

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='output',data=heart,palette='RdBu_r')

In [ ]:
heart['output'].value_counts(normalize=True)*100

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='output',hue='sex',data=heart,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='output',hue='cp',data=heart,palette='rainbow')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='output',hue='restecg',data=heart,palette='rainbow')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='output',hue='exng',data=heart,palette='rainbow')

In [ ]:
sns.distplot(heart[heart.output==0]['age'].dropna(),color='red')
sns.distplot(heart[heart.output==1]['age'].dropna(),color='green')

In [ ]:
heart[['cp','output','age']].groupby(['cp','output']).count()

In [ ]:
heart[['thall','output','age']].groupby(['thall','output']).count()

---

# Training and Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(heart.drop(['output'],axis=1), 
                                                    heart['output'], test_size=0.20, 
                                                    random_state=101)

In [ ]:
lr = LogisticRegression()
rfc = RandomForestClassifier(n_estimators=50)
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
dt = DecisionTreeClassifier(criterion="entropy",random_state=1, max_depth=3)

In [ ]:
# Logistic Regression and Random Forest
eclf = VotingClassifier(estimators=[('lr', lr), ('rfc', rfc)], voting='soft')
eclf = eclf.fit(X_train,y_train)
predictions = eclf.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
# Random Forest
eclf = VotingClassifier(estimators=[('rf', rfc)], voting='soft')
eclf = eclf.fit(X_train,y_train)
predictions = eclf.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
# Decision Tree
eclf = VotingClassifier(estimators=[('dt', dt)], voting='soft')
eclf = eclf.fit(X_train,y_train)
predictions = eclf.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
# K-Nearest Neighbor
eclf = VotingClassifier(estimators=[('knn', knn)], voting='soft')
eclf = eclf.fit(X_train,y_train)
predictions = eclf.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
# Logistic Regression
eclf = VotingClassifier(estimators=[('lr', lr)], voting='soft')
eclf = eclf.fit(X_train,y_train)
predictions = eclf.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
#K-Means
km = KMeans(n_clusters=2)
km.fit(heart.drop('output',axis=1))
km.cluster_centers_

In [ ]:
cluster = km.labels_
heart['Cluster']=heart['output'].apply(lambda x: 1 if x==1 else 0)

In [ ]:
print(confusion_matrix(heart['Cluster'],km.labels_))
print('\n')
print(classification_report(heart['Cluster'],km.labels_))

# Random Forrest Classifier gives the best result with 0.87 accuracy